In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import datasets
import torchvision

In [2]:
train_path = r'D:\Datasets\gtsrb\gtsrb'

In [3]:
transform =  transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [50]:
train_data = datasets.ImageFolder(train_path, transform=transform)
trainloader = torch.utils.data.DataLoader(
    train_data,
    batch_size=2000,
    shuffle=True,
    num_workers=0,
    pin_memory = True
)


In [51]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16*6*6, 64)
        self.fc2 = nn.Linear(64, 131)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*6*6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda:0")
net = Model().to(device)

In [52]:
import torch.optim as optim
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters())

In [53]:
from torch.autograd import Variable
for epoch in range(10):
    for i, data in enumerate(trainloader):
        inputs, labels = Variable(data[0].cuda()), Variable(data[1].cuda())
        opt.zero_grad()
        out = net(inputs)
        loss = loss_fn(out, labels)
        loss.backward()
        opt.step()
    print(loss.item())

KeyboardInterrupt: 

In [40]:
for x, y in trainloader:
    pred = net(x.cuda())
    _, pre = torch.max(pred, 1)
    c = (pre == y.cuda()).sum()
    print(c.cpu().numpy()/len(y))
    

0.5775
0.6
0.5875
0.583
0.594
0.5965
0.585
0.5855
0.5935
0.5825
0.564
0.574
0.5755
0.6078125
